<a href="https://colab.research.google.com/github/AmirHDehghan/Projects/blob/main/Papers_Implemented/Unsupervised%D9%80UShapelets/Unsupervised%D9%80UShapelets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore

In [ ]:
def Compute_distance(candidate_shapelets, dataset):
  distance = np.array([])
  candidate_shapelets = zscore(candidate_shapelets) #z_normalize the candidate_shapelets
  for lend in range(len(dataset.shape[0])):
    time_s = dataset[lend, :]
    distancel = float("inf")
    for posn in range(len(time_s) - len(candidate_shapelets) + 1):
      z = zscore(time_s[posn * len(candidate_shapelets)])
      d = np.linalg.norm(z - candidate_shapelets)
      distance = np.append(distance, min(d, distancel))
  return distance/np.sqrt(len(candidate_shapelets))

In [ ]:
def Compute_gap(candidate_shapelets, dataset, cluster_num):
  distance = Compute_distance(candidate_shapelets, dataset)
  distance = np.sort(distance)
  max_gap = 0;
  dt = 0

  for d_loc in range(1, len(distance)):
    d = (distance[d_loc] + distance[d_loc + 1])/2
    d_a = distance[np.where(distance < d)]
    d_b = distance[np.where(distance > d)]
    d_ratio = len(d_a)/len(d_b)
    if (cluster_num < d_ratio < (1-1/cluster_num)):     # K: number of clusters
      ua = np.mean(distance[d_a])
      ub = np.mean(distance[d_b])
      sta = np.std(distance[d_a])
      stb = np.std(distance[d_b])

      gap = ub - stb - (ua + sta)

      if gap > max_gap:
        max_gap = gap
        dt = d

  return max_gap, dt

In [ ]:
def Extract_UShapelets(dataset, shapelets_len):
  
  set_of_u_shapelets = []
  time_series = dataset[0,:]

  while True:
    set_of_subsequences = np.array([])
    count = 0
    gap = np.array([])
    dt = np.array([])

    for slen in range(shapelets_len):
      for win in range(len(time_series) - slen + 1):
        set_of_subsequences = np.append(set_of_subsequences ,time_series[win:win+slen -1])
        # Compute Gap 
        gapp, dtt = Compute_gap(candidate_shapelets= set_of_subsequences[count], dataset= dataset, cluster_num= 5)
        gap = np.append(gap, gapp)
        dt = np.append(dt, dtt)
        count +=1
    index1 = np.where(gap = max(gap))[0]
    set_of_u_shapelets = np.append(set_of_u_shapelets, set_of_subsequences(index1))
    # Compute Distance
    distance = Compute_distance(set_of_subsequences(index1), dataset)

    d_a = distance[np.where(distance < dt)[0]]

    if len(d_a) == 1:
      break
    else:
      index2 = np.where(distance = max(distance))[0]
      time_series = dataset[index2,:]
      theta = np.mean(Compute_distance(d_a, dataset)) +  np.std(Compute_distance(d_a, dataset))
      dataset = np.delete(dataset, np.where(distance < theta)[0])
  return set_of_u_shapelets